# Tutorial 3: Mounting LightyNode to TransportPCE

## Learning Objectives
- Understand device mounting concept in SDN controllers
- Mount LightyNode devices to TransportPCE using RESTCONF
- Verify device connectivity and status
- Monitor device health through the controller
- View devices in network topology

## Prerequisites
- Completed Tutorials 1-2
- Running LightyNode containers and TransportPCE controller

## Overview
Device mounting establishes management connections between the SDN controller and network devices. TransportPCE uses NETCONF internally to communicate with devices, but you interact with the controller using RESTCONF APIs.

## Step 1: Student Configuration

Configure your student ID for isolated environment setup.

In [ ]:
# Student Configuration
STUDENT_ID = "1"  # Change this to your assigned student ID

NETWORK_NAME = f"student{STUDENT_ID}-network"
TPCE_CONTAINER_NAME = f"student{STUDENT_ID}-tpce"

print(f"Student ID: {STUDENT_ID}")
print(f"Network: {NETWORK_NAME}")
print(f"TransportPCE Container: {TPCE_CONTAINER_NAME}")

## Step 2: Import Required Libraries

Import necessary libraries for container management and RESTCONF operations.

In [ ]:
# Import libraries
import docker
import requests
import json
import time
from requests.auth import HTTPBasicAuth
from common_utils import StatusIndicators, print_status, print_section, RESTClient, DockerHelper

docker_helper = DockerHelper()

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print_status("Libraries imported successfully", StatusIndicators.SUCCESS)

## Step 3: Define Helper Functions

Create functions to manage device mounting and network operations.

In [ ]:
# Helper functions

def get_container_ip(container_name):
    """Get container IP address"""
    return docker_helper.get_container_ip(container_name, NETWORK_NAME)


def mount_device_to_tpce(device_id, device_ip, show_details=True):
    """Mount a device to TransportPCE"""
    tpce_ip = get_container_ip(TPCE_CONTAINER_NAME)
    if not tpce_ip:
        print_status("Cannot get TransportPCE IP", StatusIndicators.ERROR)
        return False

    mount_payload = {
        "node": {
            "node-id": device_id,
            "netconf-node-topology:netconf-node": {
                "netconf-node-topology:host": device_ip,
                "netconf-node-topology:port": 830,
                "netconf-node-topology:login-password-unencrypted": {
                    "netconf-node-topology:username": "admin",
                    "netconf-node-topology:password": "admin"
                },
                "netconf-node-topology:tcp-only": "false",
                "netconf-node-topology:reconnect-on-changed-schema": "false",
                "netconf-node-topology:connection-timeout-millis": 20000,
                "netconf-node-topology:max-connection-attempts": 5,
                "netconf-node-topology:keepalive-delay": 120,
                "netconf-node-topology:yang-module-capabilities": {
                    "capability": [
                        "urn:ietf:params:xml:ns:netmod:notification?module=nc-notifications&revision=2008-07-14",
                        "http://org/openroadm/ethernet-interfaces?module=org-openroadm-ethernet-interfaces&revision=2020-05-29"
                    ],
                    'override': False
                }
            }
        }
    }

    url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}"
    headers = {'Content-Type': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')

    # 打印 Mount Payload
    print_status(f"Preparing to mount {device_id} at {device_ip}", StatusIndicators.INFO)
    print("\n" + "="*80)
    print("MOUNT PAYLOAD:")
    print("="*80)
    print(json.dumps(mount_payload, indent=2))
    print("="*80 + "\n")

    if show_details:
        print(f"URL: {url}")
        print(f"Method: PUT")
        print(f"Headers: {headers}")
        print(f"Auth: admin/admin\n")

    try:
        response = requests.put(url, json=mount_payload, headers=headers, auth=auth, timeout=30)

        if show_details:
            print(f"\nMount Response:")
            print(f"Status: {response.status_code}")
            print(f"Headers: {dict(response.headers)}")
            print(f"Body: {response.text}")

        success = response.status_code in [200, 201, 204]
        if success:
            print_status(f"Successfully mounted {device_id}", StatusIndicators.SUCCESS)
        else:
            print_status(f"Failed to mount {device_id}", StatusIndicators.ERROR)

        return success
    except Exception as e:
        print_status(f"Mount error for {device_id}", StatusIndicators.ERROR)
        print(f"Error: {e}")
        return False

def get_device_status(device_id, show_details=True):
    """Gets the connection status of a device from TransportPCE"""
    tpce_ip = get_container_ip(TPCE_CONTAINER_NAME)
    if not tpce_ip:
        return None

    url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}?content=nonconfig"
    headers = {'Accept': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')

    try:
        response = requests.get(url, headers=headers, auth=auth, timeout=10)

        if show_details:
            print(f"\nDevice Status Response for {device_id}:")
            print(f"Status: {response.status_code}")
            print(f"Headers: {dict(response.headers)}")

        if response.ok:
            data = response.json()
            if show_details:
                print(f"Body: {json.dumps(data, indent=2)}")

            # Parse connection status
            if 'network-topology:node' in data:
                node_data = data['network-topology:node'][0]
                netconf_node = node_data.get('netconf-node-topology:netconf-node', {})
                connection_status = (
                        node_data.get("netconf-node-topology:connection-status") or
                        netconf_node.get('connection-status') or
                        "unknown"
                )
            else:
                connection_status = "device-not-found"

            if show_details:
                print_status(f"{device_id} status: {connection_status}",
                             StatusIndicators.SUCCESS if connection_status == 'connected' else StatusIndicators.INFO)

            return connection_status
        else:
            if show_details:
                print(f"Body: {response.text}")
            return {"error": f"HTTP {response.status_code}"}

    except Exception as e:
        if show_details:
            print(f"Error: {e}")
        return {"error": str(e)}


print_status("Helper functions defined", StatusIndicators.SUCCESS)

## Step 4: Initialize TransportPCE Connection

Get the TransportPCE container IP address for RESTCONF operations.

In [ ]:
# Get TransportPCE IP
tpce_ip = get_container_ip(TPCE_CONTAINER_NAME)

if tpce_ip:
    print(f"TransportPCE IP: {tpce_ip}")
else:
    print("Could not get TransportPCE IP - ensure Tutorial 2 completed successfully")

## Step 5: Discover LightyNode Devices

Find all LightyNode containers in the student network for mounting.

In [ ]:
# Discover LightyNode containers
if tpce_ip:
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        network_info = docker_helper.client.api.inspect_network(network.id)
        containers = network_info.get('Containers', {})

        lightynode_containers = []
        for container_info in containers.values():
            name = container_info.get('Name', '')
            ip = container_info.get('IPv4Address', '').split('/')[0]

            if 'roadm' in name.lower() or 'tpdr' in name.lower():
                device_type = 'ROADM' if 'roadm' in name.lower() else 'TRANSPONDER'
                device_id = name.replace('_LightyNode-simulator', '').replace(f'student{STUDENT_ID}-', '')

                lightynode_containers.append({
                    'device_id': device_id,
                    'ip_address': ip,
                    'device_type': device_type
                })

        print(f"Found {len(lightynode_containers)} LightyNode devices:")
        for device in lightynode_containers:
            print(f"  • {device['device_id']} ({device['device_type']}) at {device['ip_address']}")

    except Exception as e:
        print(f"Error discovering devices: {e}")
        lightynode_containers = []
else:
    lightynode_containers = []

## Step 6: Demonstration - Mount ROADM Device

We'll demonstrate the mounting process by mounting the ROADM device first. You'll then practice with the transponder.

In [ ]:
# Demonstrate mounting with ROADM device
if lightynode_containers and tpce_ip:
    print("=== DEMONSTRATION: MOUNTING ROADM DEVICE ===")

    # Find the ROADM device
    roadm_device = next((d for d in lightynode_containers if d['device_type'] == 'ROADM'), None)

    if roadm_device:
        print(f"\nDemonstrating mount process with ROADM device:")
        print(f"Device ID: {roadm_device['device_id']}")
        print(f"Device Type: {roadm_device['device_type']}")
        print(f"IP Address: {roadm_device['ip_address']}")

        print(f"\nStep 1: Using mount_device_to_tpce() helper function")
        print("This function handles:")
        print("  • Creating the complete NETCONF node topology payload")
        print("  • Setting up authentication and connection parameters")
        print("  • Sending the mounting request to TransportPCE")
        print("  • Displaying detailed operation results")

        print(f"\nStep 2: Calling mount function")
        success = mount_device_to_tpce(roadm_device['device_id'], roadm_device['ip_address'], show_details=True)

        if success:
            print(f"\nROADM mount request successful!")
            print(f"The ROADM device is now being mounted to TransportPCE")
        else:
            print(f"\nROADM mount request failed")

        print(f"\nStep 3: Wait for connection to establish...")
        time.sleep(5)  # Wait for mounting to complete

        # Display recent TransportPCE logs (simplified inline)
        print(f"\nTransportPCE Recent Logs:")
        try:
            tpce_container = docker_helper.client.containers.get(TPCE_CONTAINER_NAME)
            logs = tpce_container.logs(tail=5).decode('utf-8', errors='ignore')
            for i, line in enumerate(logs.split('\n'), 1):
                if line.strip():
                    print(f"  {i} | {line[:1000]}")
        except Exception as e:
            print(f"  Could not retrieve logs: {e}")

    else:
        print("No ROADM device found for demonstration")

else:
    print("Cannot demonstrate mounting - missing TransportPCE or no devices found")

## Step 7: Demonstration - Check ROADM Device Status

Now we'll demonstrate how to check the individual device connection status using the TransportPCE developer guide approach.

In [ ]:
# Demonstration: Check ROADM device connection status
if lightynode_containers and tpce_ip:
    print("=== DEMONSTRATION: CHECKING ROADM DEVICE STATUS ===")

    # Find the ROADM device
    roadm_device = next((d for d in lightynode_containers if d['device_type'] == 'ROADM'), None)

    if roadm_device:
        device_id = roadm_device['device_id']
        device_type = roadm_device['device_type']

        print(f"\nDevice to check: {device_id} ({device_type})")

        # Build the RESTCONF URL for device status
        url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}?content=nonconfig"
        headers = {'Accept': 'application/json'}
        auth = HTTPBasicAuth('admin', 'admin')

        print(f"\nSending GET request to: {url}")

        try:
            # Make the request
            response = requests.get(url, headers=headers, auth=auth, timeout=10)

            # Show raw response
            print(f"\nResponse Status: {response.status_code}")

            if response.ok:
                data = response.json()
                print(f"\nRaw Response:")
                print(json.dumps(data, indent=2))

                # Extract connection status from response
                if 'network-topology:node' in data:
                    node_data = data['network-topology:node'][0]
                    # Check for connection status in different possible locations
                    connection_status = (
                            node_data.get("netconf-node-topology:connection-status") or
                            node_data.get("netconf-node-topology:netconf-node", {}).get("connection-status") or
                            "unknown"
                    )
                else:
                    connection_status = "not-found"

                print(f"\nExtracted Connection Status: {connection_status}")

                # Interpret the status
                if connection_status == "connected":
                    print_status("Device is successfully connected to TransportPCE", StatusIndicators.SUCCESS)
                elif connection_status == "connecting":
                    print_status("Device is still connecting...", StatusIndicators.WORKING)
                else:
                    print_status("Device is not connected", StatusIndicators.ERROR)

            else:
                print(f"Request failed with status {response.status_code}")
                print(f"Response: {response.text}")

        except Exception as e:
            print(f"Error checking device status: {e}")

        print(f"\n{'=' * 50}")
        print("ROADM STATUS CHECK COMPLETE")
        print("Next: You'll practice this with the transponder device")
        print(f"{'=' * 50}")

    else:
        print("No ROADM device found for status demonstration")

else:
    print("Cannot demonstrate status checking - missing TransportPCE or no devices found")

## EXERCISE: Mount the Transponder Device
Fill in the blanks with actual values to mount the Transponder

In [ ]:
if tpce_ip:
    print("=== EXERCISE: MOUNTING TRANSPONDER DEVICE ===")
    print("\nLook at the discovered devices above and fill in the actual values")

    # EXERCISE 1: Fill in the transponder device information
    # Hint: Look at Step 6 output for the actual values
    device_id = "___"    # Fill in: "tpdr-a" or your transponder name
    device_ip = "___"    # Fill in: The IP address shown in Step 5

    print(f"\nDevice to mount:")
    print(f"  Device ID: {device_id}")
    print(f"  Device IP: {device_ip}")

    # EXERCISE 2: Build the mounting payload
    print("\nBuilding NETCONF mounting payload...")
    print("Fill in the blanks in the payload below:")

    mount_payload = {
        "node": {
            "node-id": device_id,  # Uses the device_id from above
            "netconf-node-topology:netconf-node": {
                "netconf-node-topology:host": device_ip,  # Uses the device_ip from above
                "netconf-node-topology:port": ___,  # Fill in: NETCONF port (hint: 830)
                "netconf-node-topology:login-password-unencrypted": {
                    "netconf-node-topology:username": "___",  # Fill in: "admin"
                    "netconf-node-topology:password": "___"   # Fill in: "admin"
                },
                "netconf-node-topology:tcp-only": "false",
                "netconf-node-topology:reconnect-on-changed-schema": "false",
                "netconf-node-topology:connection-timeout-millis": 20000,
                "netconf-node-topology:max-connection-attempts": 5,
                "netconf-node-topology:keepalive-delay": 120
            }
        }
    }

    # EXERCISE 3: Send the mounting request
    print("\nSending mount request to TransportPCE...")

    # Build the URL
    url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}"

    # Set up request parameters
    headers = {'Content-Type': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')

    print(f"URL: {url}")
    print(f"Method: PUT")  # We use PUT to create/update the device mount

    try:
        # EXERCISE 4: Make the HTTP request
        # Fill in the HTTP method: "put", "post", or "get"?
        response = requests._____(url, json=mount_payload, headers=headers, auth=auth, timeout=30)

        print(f"\nResponse Status: {response.status_code}")

        if response.status_code in [200, 201, 204]:
            print_status("Mount request successful!", StatusIndicators.SUCCESS)

            # Wait and check logs
            print("\nWaiting for device to connect...")
            time.sleep(5)

            # Show simple logs
            try:
                tpce_container = docker_helper.client.containers.get(TPCE_CONTAINER_NAME)
                logs = tpce_container.logs(tail=3).decode('utf-8', errors='ignore')
                print("\nRecent TransportPCE logs:")
                for line in logs.split('\n'):
                    if line.strip():
                        print(f"  {line[:100]}")
            except:
                pass

        else:
            print_status(f"Mount failed with status {response.status_code}", StatusIndicators.ERROR)
            if response.text:
                print(f"Error: {response.text}")

    except Exception as e:
        print(f"Error mounting device: {e}")

    print("\n" + "="*50)
    print("EXERCISE COMPLETE")
    print("You learned how to:")
    print("  • Build a NETCONF mounting payload")
    print("  • Use RESTCONF PUT to mount a device")
    print("  • Check logs to verify mounting")
    print("="*50)

else:
    print("Cannot run exercise - TransportPCE not available")

## SOLUTION: Mount the Transponder Device

In [ ]:
if tpce_ip and lightynode_containers:
    print("=== SOLUTION: MOUNTING TRANSPONDER DEVICE ===")

    # Find transponder from discovered devices
    xpdr_device = next((d for d in lightynode_containers if d['device_type'] == 'TRANSPONDER'), None)

    if xpdr_device:
        # Extract actual values
        device_id = xpdr_device['device_id']  # e.g., "tpdr-a"
        device_ip = xpdr_device['ip_address']  # e.g., "172.19.0.3"

        print(f"\nDevice to mount:")
        print(f"  Device ID: {device_id}")
        print(f"  Device IP: {device_ip}")

        # Build the complete mounting payload
        mount_payload = {
            "node": {
                "node-id": device_id,
                "netconf-node-topology:netconf-node": {
                    "netconf-node-topology:host": device_ip,
                    "netconf-node-topology:port": 830,  # NETCONF standard port
                    "netconf-node-topology:login-password-unencrypted": {
                        "netconf-node-topology:username": "admin",
                        "netconf-node-topology:password": "admin"
                    },
                    "netconf-node-topology:tcp-only": "false",
                    "netconf-node-topology:reconnect-on-changed-schema": "false",
                    "netconf-node-topology:connection-timeout-millis": 20000,
                    "netconf-node-topology:max-connection-attempts": 5,
                    "netconf-node-topology:keepalive-delay": 120
                }
            }
        }

        # Send the mounting request
        url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}"
        headers = {'Content-Type': 'application/json'}
        auth = HTTPBasicAuth('admin', 'admin')

        print(f"\nSending PUT request to: {url}")
        print(f"Payload: {json.dumps(mount_payload, indent=2)}")

        try:
            response = requests.put(url, json=mount_payload, headers=headers, auth=auth, timeout=30)

            print(f"\nResponse Status: {response.status_code}")

            if response.status_code in [200, 201, 204]:
                print_status("Transponder mounted successfully!", StatusIndicators.SUCCESS)

                # Wait and check logs
                print("\nWaiting for device to connect...")
                time.sleep(5)

                # Show simple logs
                try:
                    tpce_container = docker_helper.client.containers.get(TPCE_CONTAINER_NAME)
                    logs = tpce_container.logs(tail=3).decode('utf-8', errors='ignore')
                    print("\nRecent TransportPCE logs:")
                    for line in logs.split('\n'):
                        if line.strip():
                            print(f"  {line[:1000]}")
                except:
                    pass

            else:
                print_status(f"Mount failed with status {response.status_code}", StatusIndicators.ERROR)
                if response.text:
                    print(f"Error: {response.text}")

        except Exception as e:
            print(f"Error mounting device: {e}")
    else:
        print("No transponder device found in network")

    print("\n" + "="*50)
    print("SOLUTION COMPLETE")
    print("The transponder is now mounted to TransportPCE")
    print("="*50)

else:
    print("Cannot run solution - missing prerequisites")

## Step 9: Exercise - Check Transponder Device Status

**EXERCISE**: Now it's your turn to check device status! You've seen how to check the ROADM device status. Complete the code below to check the transponder (XPDR) device connection status.

**Learning Goals:**
- Practice device status checking operations
- Complete RESTCONF status query requests
- Understand connection status interpretation
- Troubleshoot device connectivity issues

In [ ]:
# EXERCISE: Check Transponder Device Status
# Fill in the blanks with actual values to check device status

if tpce_ip:
    print("=== EXERCISE: CHECKING TRANSPONDER DEVICE STATUS ===")
    print("\nFill in the actual values based on what you learned")
    
    # EXERCISE 1: Fill in device information
    # Hint: Use the same values from the mounting exercise
    device_id = "___"        # Fill in: "tpdr-a" or your transponder name
    device_type = "___"      # Fill in: "TRANSPONDER"
    
    print(f"\nDevice to check:")
    print(f"  Device ID: {device_id}")
    print(f"  Device Type: {device_type}")
    
    # EXERCISE 2: Build the RESTCONF URL for status checking
    url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}?content=nonconfig"
    
    # EXERCISE 3: Set up request parameters
    # Fill in the request parameters
    headers = {'Accept': 'application/___'}  # Fill in: "json"
    auth = HTTPBasicAuth('___', '___')       # Fill in: "admin", "admin"
    
    print(f"\nSending GET request to check status...")
    print(f"URL: {url}")
    
    try:
        # EXERCISE 4: Make the HTTP request
        # Fill in the HTTP method for reading data
        response = requests._____(url, headers=headers, auth=auth, timeout=10)  # Fill in: "get"
        
        print(f"\nResponse Status: {response.status_code}")
        
        if response.ok:
            data = response.json()
            
            # Show raw response (educational)
            print("\nRaw JSON Response:")
            print(json.dumps(data, indent=2))
            
            # Extract connection status
            if 'network-topology:node' in data:
                node_data = data['network-topology:node'][0]
                connection_status = (
                    node_data.get("netconf-node-topology:connection-status") or
                    node_data.get("netconf-node-topology:netconf-node", {}).get("connection-status") or
                    "unknown"
                )
            else:
                connection_status = "not-found"
            
            print(f"\nExtracted Connection Status: {connection_status}")
            
            # Simple interpretation
            if connection_status == "connected":
                print("✓ Device is connected and ready")
            elif connection_status == "connecting":
                print("⋯ Device is still connecting...")
            else:
                print("✗ Device is not connected")
                
        else:
            print(f"Request failed: {response.status_code}")
            if response.text:
                print(f"Error: {response.text}")
                
    except Exception as e:
        print(f"Error checking status: {e}")
    
    print("\n" + "="*50)
    print("EXERCISE COMPLETE")
    print("You learned how to:")
    print("  • Build RESTCONF URLs for status queries")
    print("  • Use GET requests to read device status")
    print("  • Parse JSON responses to find connection status")
    print("="*50)
    
else:
    print("Cannot run exercise - TransportPCE not available")

## Solution

Here's the complete solution with all blanks filled in:

In [ ]:
# EXERCISE: Check Transponder Device Status
# Fill in the blanks with actual values to check device status

if tpce_ip:
    print("=== EXERCISE: CHECKING TRANSPONDER DEVICE STATUS ===")
    print("\nFill in the actual values based on what you learned")

    # EXERCISE 1: Fill in device information
    # Hint: Use the same values from the mounting exercise
    device_id = "tpdr-a"      # e.g., "tpdr-a"
    device_type = "TRANSPONDER"      # Fill in: "TRANSPONDER"

    print(f"\nDevice to check:")
    print(f"  Device ID: {device_id}")
    print(f"  Device Type: {device_type}")

    # EXERCISE 2: Build the RESTCONF URL for status checking
    url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}?content=nonconfig"

    # EXERCISE 3: Set up request parameters
    # Fill in the request parameters
    headers = {'Accept': 'application/json'}  # Fill in: "json"
    auth = HTTPBasicAuth('admin', 'admin')       # Fill in: "admin", "admin"

    print(f"\nSending GET request to check status...")
    print(f"URL: {url}")

    try:
        # EXERCISE 4: Make the HTTP request
        # Fill in the HTTP method for reading data
        response = requests.get(url, headers=headers, auth=auth, timeout=10)  # Fill in: "get"

        print(f"\nResponse Status: {response.status_code}")

        if response.ok:
            data = response.json()

            # Show raw response (educational)
            print("\nRaw JSON Response:")
            print(json.dumps(data, indent=2))

            # Extract connection status
            if 'network-topology:node' in data:
                node_data = data['network-topology:node'][0]
                connection_status = (
                    node_data.get("netconf-node-topology:connection-status") or
                    node_data.get("netconf-node-topology:netconf-node", {}).get("connection-status") or
                    "unknown"
                )
            else:
                connection_status = "not-found"

            print(f"\nExtracted Connection Status: {connection_status}")

            # Simple interpretation
            if connection_status == "connected":
                print("✓ Device is connected and ready")
            elif connection_status == "connecting":
                print("⋯ Device is still connecting...")
            else:
                print("✗ Device is not connected")

        else:
            print(f"Request failed: {response.status_code}")
            if response.text:
                print(f"Error: {response.text}")

    except Exception as e:
        print(f"Error checking status: {e}")

    print("\n" + "="*50)
    print("EXERCISE COMPLETE")
    print("You learned how to:")
    print("  • Build RESTCONF URLs for status queries")
    print("  • Use GET requests to read device status")
    print("  • Parse JSON responses to find connection status")
    print("="*50)

else:
    print("Cannot run exercise - TransportPCE not available")

## Step 10: Summary

In this tutorial, you have successfully:

### **Device Mounting Concepts**
- **Understood SDN device mounting** - How controllers establish connections with devices
- **NETCONF Protocol** - Learned that TransportPCE uses NETCONF internally (port 830)
- **RESTCONF Interface** - Used HTTP/JSON to manage device mounting operations
- **Topology Management** - Observed how devices appear in controller topology views

### **Device Status Checking**
- **Individual Device Queries** - Used TransportPCE developer guide approach for status checking
- **Connection State Analysis** - Interpreted 'connected', 'connecting', 'unable-to-connect' states
- **Branch Version Handling** - Learned how different TransportPCE versions structure responses
- **Troubleshooting Skills** - Developed systematic approach to diagnosing connection issues

### **Hands-On Skills Developed**
- **Device Discovery** - Found LightyNode containers in student network
- **Mounting Operations** - Demonstrated ROADM mounting process with correct payload structure
- **Interactive Practice** - Completed transponder mounting exercise
- **Status Verification** - Used get_device_status() function for detailed device checking
- **Status Interpretation** - Analyzed connection status and provided troubleshooting guidance

### **Technical Understanding**
- **Mount Payload Structure** - Complete NETCONF node topology payload with capabilities
- **Connection States** - Deep understanding of device lifecycle during mounting
- **Multiple Topologies** - NETCONF topology vs OpenROADM topology differences  
- **API Patterns** - RESTCONF GET vs PUT operations for different purposes
- **Error Handling** - Proper timeout and error checking for network operations

### **Problem-Solving Experience**
- **Fill-in-the-Blanks Exercises** - Practiced completing real API calls for both mounting and status checking
- **Request Construction** - Built proper URLs, headers, and payloads for different operations
- **Response Analysis** - Interpreted HTTP status codes and JSON responses from different endpoints
- **Network Debugging** - Verified device connectivity and developed troubleshooting workflows

## Key Functions Mastered

**Device Mounting Process:**
1. Discover device (IP address, credentials)
2. Create mounting payload (complete NETCONF node configuration)
3. Send RESTCONF PUT request to controller
4. Wait for device to establish NETCONF connection
5. Verify device appears in topology

**Device Status Checking Process:**
1. Query device status via RESTCONF GET with `?content=nonconfig`
2. Parse response based on TransportPCE branch version
3. Extract connection-status from nested JSON structure
4. Interpret status and provide actionable feedback
5. Display formatted status summary for user

**Critical Parameters:**
- **Node ID**: Unique identifier for device in controller
- **Host/IP**: Network address of the device  
- **Port 830**: Standard NETCONF port
- **Credentials**: admin/admin (default for LightyNode)
- **TCP-only**: False (enables SSH for NETCONF)
- **Capabilities**: OpenROADM YANG modules for device functionality

## New Functions Available

**get_device_status(device_id, tpce_branch='master', show_details=True)**
- Queries individual device connection status
- Handles multiple TransportPCE branch versions
- Returns connection status string or error information

**display_device_status_summary(device_id, device_type, connection_status)**
- Formats device status in user-friendly display
- Provides status icons and descriptions
- Includes troubleshooting recommendations

**display_container_logs(container_name, tail_lines=5, title="Container Logs")**
- Shows recent container logs for troubleshooting
- Helps understand what's happening during mounting process

## What You've Built

Your network environment now includes:
- **Student-isolated network** with unique containers
- **TransportPCE controller** managing multiple devices with proper payload structure
- **ROADM device** mounted and status-checked (demonstrated)
- **Transponder device** mounted and status-checked (your exercise)
- **Network topology** visible through controller APIs
- **Status monitoring capabilities** for ongoing device health checking
- **Log viewing capabilities** for troubleshooting connection issues

## Ready for Next Steps

With both devices now mounted to TransportPCE and status checking capabilities in place, you're prepared for:
- **Interface creation** on mounted devices using OpenROADM models
- **Service provisioning** between devices with end-to-end lightpaths
- **Network topology building** with links and connections
- **Continuous monitoring** of device health using status functions
- **Advanced troubleshooting** of NETCONF connectivity issues

### **Advanced Skills Unlocked**
- Individual device status monitoring and troubleshooting
- TransportPCE API expertise for device lifecycle management  
- NETCONF/RESTCONF protocol understanding
- OpenROADM device integration patterns
- Container log analysis for debugging connection issues